In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping

2025-07-25 10:15:25.883548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753438526.229371      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753438526.362438      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv") 
print("Data Shape:", df.shape)
print("Class Distribution:\n", df["Class"].value_counts())

Data Shape: (284807, 31)
Class Distribution:
 Class
0    284315
1       492
Name: count, dtype: int64


In [3]:
X = df.drop("Class", axis=1)
y = df["Class"]

In [4]:
scaler = StandardScaler()
X[["Time", "Amount"]] = scaler.fit_transform(X[["Time", "Amount"]])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [6]:
import joblib
joblib.dump(scaler, "fraud_scaler.bin")


['fraud_scaler.bin']

In [7]:
fraud_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
class_weights = {0: 1, 1: fraud_weight}
print(f"Class weights: {class_weights}")

Class weights: {0: 1, 1: 577.2868020304569}


In [8]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # 30 features
    Dense(64, activation='relu'),
    Dropout(0.3),  # Reduce overfitting
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Fraud probability output
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
print(model.summary())

2025-07-25 10:15:47.942597: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,097 (16.00 KB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - auc: 0.7662 - loss: 0.0213 - precision: 0.5861 - recall: 0.4305 - val_auc: 0.9333 - val_loss: 0.0035 - val_precision: 0.8481 - val_recall: 0.8072
Epoch 2/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - auc: 0.9290 - loss: 0.0041 - precision: 0.8208 - recall: 0.7365 - val_auc: 0.9394 - val_loss: 0.0031 - val_precision: 0.8767 - val_recall: 0.7711
Epoch 3/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - auc: 0.9301 - loss: 0.0033 - precision: 0.8845 - recall: 0.7598 - val_auc: 0.9335 - val_loss: 0.0032 - val_precision: 0.8800 - val_recall: 0.7952
Epoch 4/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - auc: 0.9372 - loss: 0.0034 - precision: 0.8575 - recall: 0.7405 - val_auc: 0.9216 - val_loss: 0.0038 - val_precision: 0.8800 - val_recall: 0.7952
Epoch 5/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - auc: 0.9486 - loss: 0.0030 - precision: 0.8596 - recall: 0.7431 - val_auc: 0.9275 - val_loss: 0.0030 - val_precision: 0.8684 - val_

In [14]:
model.save('fraud_detection_model.keras')